# In this notebook, we going to create a sentimental feature for filtering

## 1. Loading the data

In [2]:
import pandas as pd

books = pd.read_csv('books_with_categories.csv')

## 2. We will use a sentimental model for this job

### The model will predict a document/query into 7 shades

In [6]:
from transformers import pipeline
import torch
classifier = pipeline("text-classification",
                      model="j-hartmann/emotion-english-distilroberta-base",
                      device = 0 if torch.cuda.is_available() else -1,
                      top_k=None,)
classifier("I love programming!")

Device set to use cpu


[[{'label': 'joy', 'score': 0.9866405129432678},
  {'label': 'sadness', 'score': 0.003636218374595046},
  {'label': 'surprise', 'score': 0.003231387585401535},
  {'label': 'neutral', 'score': 0.0027272505685687065},
  {'label': 'anger', 'score': 0.0026277839206159115},
  {'label': 'disgust', 'score': 0.0007785824709571898},
  {'label': 'fear', 'score': 0.0003582451317925006}]]

In [7]:
books["description"][0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [8]:
classifier(books["description"][0])

[[{'label': 'fear', 'score': 0.6548399925231934},
  {'label': 'neutral', 'score': 0.1698525995016098},
  {'label': 'sadness', 'score': 0.11640939861536026},
  {'label': 'surprise', 'score': 0.02070068009197712},
  {'label': 'disgust', 'score': 0.019100721925497055},
  {'label': 'joy', 'score': 0.015161462128162384},
  {'label': 'anger', 'score': 0.003935154061764479}]]

In [10]:
sentences = books["description"][0].split(".")
predictions = classifier(sentences)

#### But the model may miss some infomation like the meaning of the whole description can be interpret as suprise for a human but the sentence 3 is classifed as fear and take the highest score

#### So we gonna classify the sentiment for all the sentence in a description use all the sentimental shades as features

In [12]:
for i in range(len(sentences)):
    print(f"Sentence: {sentences[i]}")
    print(f"Predictions: {predictions[i]}")
    print("\n")

Sentence: A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives
Predictions: [{'label': 'surprise', 'score': 0.7296021580696106}, {'label': 'neutral', 'score': 0.14038598537445068}, {'label': 'fear', 'score': 0.06816227734088898}, {'label': 'joy', 'score': 0.04794260486960411}, {'label': 'anger', 'score': 0.009156367741525173}, {'label': 'disgust', 'score': 0.0026284768246114254}, {'label': 'sadness', 'score': 0.002122163772583008}]


Sentence:  John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers
Predictions: [{'label': 'neutral', 'score': 0.4493706524372101}, {'label': 'disgust', 'score': 0.27359113097190857}, {'label': 'joy', 'score': 0.1090831384062767}, {'label': 'sadness', 'score': 0.09362747520208359}, {'label': 'anger', 'score': 0.04047833010554314}, {'label': 'surprise', 'score': 0.02697019837796688}, {'label': 'fear', 'score': 0.00687

In [13]:
sorted(predictions[0], key=lambda x: x['label'])

[{'label': 'anger', 'score': 0.009156367741525173},
 {'label': 'disgust', 'score': 0.0026284768246114254},
 {'label': 'fear', 'score': 0.06816227734088898},
 {'label': 'joy', 'score': 0.04794260486960411},
 {'label': 'neutral', 'score': 0.14038598537445068},
 {'label': 'sadness', 'score': 0.002122163772583008},
 {'label': 'surprise', 'score': 0.7296021580696106}]

In [16]:
import numpy as np

emotion_labels = ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']
isbn = []
emotion_scores = {label: [] for label in emotion_labels}

def calculate_max_emotion_scores(predictions):
    per_emotion_scores = {label: [] for label in emotion_labels}
    for prediction in predictions:
        sorted_predictions = sorted(prediction, key=lambda x: x['label'])
        for index, label in enumerate(emotion_labels):
            per_emotion_scores[label].append(sorted_predictions[index]['score'])
    
    return {label: np.max(scores) for label, scores in per_emotion_scores.items()}

In [17]:
for i in range(10):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

In [18]:
emotion_scores

{'anger': [np.float64(0.06413363665342331),
  np.float64(0.6126194596290588),
  np.float64(0.06413363665342331),
  np.float64(0.35148516297340393),
  np.float64(0.0814124271273613),
  np.float64(0.23222433030605316),
  np.float64(0.5381842255592346),
  np.float64(0.06413363665342331),
  np.float64(0.3006698489189148),
  np.float64(0.06413363665342331)],
 'disgust': [np.float64(0.27359113097190857),
  np.float64(0.3482842445373535),
  np.float64(0.10400673747062683),
  np.float64(0.1507222205400467),
  np.float64(0.18449550867080688),
  np.float64(0.7271752953529358),
  np.float64(0.15585486590862274),
  np.float64(0.10400673747062683),
  np.float64(0.27948102355003357),
  np.float64(0.17792704701423645)],
 'fear': [np.float64(0.9281682968139648),
  np.float64(0.942527711391449),
  np.float64(0.9723207950592041),
  np.float64(0.36070549488067627),
  np.float64(0.09504342079162598),
  np.float64(0.051362838596105576),
  np.float64(0.7474275231361389),
  np.float64(0.4044971764087677),
  

In [19]:
from tqdm import tqdm

emotion_labels = ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']
isbn = []
emotion_scores = {label: [] for label in emotion_labels}

for i in tqdm(range(len(books))):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

100%|██████████| 5088/5088 [11:19<00:00,  7.48it/s]


In [20]:
emotions_df = pd.DataFrame(emotion_scores)
emotions_df['isbn13'] = isbn
emotions_df.head()

,anger,disgust,fear,joy,neutral,sadness,surprise,isbn13
0,0.064134,0.273591,0.928168,0.932798,0.646216,0.967158,0.729602,9780002005883
1,0.612619,0.348284,0.942528,0.704422,0.887940,0.111690,0.252546,9780002261982
2,0.064134,0.104007,0.972321,0.767239,0.549477,0.111690,0.078765,9780006178736
3,0.351485,0.150722,0.360705,0.251882,0.732685,0.111690,0.078765,9780006280897
4,0.081412,0.184496,0.095043,0.040564,0.884390,0.475880,0.078765,9780006280934


In [21]:
books = pd.merge(books, emotions_df, on='isbn13')

In [22]:
books.head()

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,...,title_and_subtitle,tagged_description,simple_categories,anger,disgust,fear,joy,neutral,sadness,surprise
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,...,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction,0.064134,0.273591,0.928168,0.932798,0.646216,0.967158,0.729602
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,...,Spider's Web A Novel,9780002261982 A new 'Christie for Christmas' -...,Fiction,0.612619,0.348284,0.942528,0.704422,0.887940,0.111690,0.252546
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,...,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction,0.064134,0.104007,0.972321,0.767239,0.549477,0.111690,0.078765
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,...,The Four Loves,9780006280897 Lewis' work on the nature of lov...,Nonfiction,0.351485,0.150722,0.360705,0.251882,0.732685,0.111690,0.078765
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,...,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",Nonfiction,0.081412,0.184496,0.095043,0.040564,0.884390,0.475880,0.078765


### Saving the data

In [23]:
books.to_csv('books_with_emotions.csv', index=False)